# This notebook is trash

In [ ]:
def f_upload_signal_awg7000(awg, filepath, wfm_name, fid_meas_log):
    # read file
    data = np.fromfile(filepath,dtype=np.uint16)
    
    # check if there is any errors on AWG's buffer
    error_str = awg.query('SYSTEM:ERROR:NEXT?')
    if error_str != '0,"No error"\n':
        fid_meas_log.write('Errors exist in AWG buffer')
    while error_str != '0,"No error"\n':
        print('AWG response to error query: ' + error_str)
        fid_meas_log.write('AWG response to error query: ' + error_str)
        # check next error
        error_str = awg.query('SYSTEM:ERROR:NEXT?')
    
    # turn AWG's output 1 OFF
    awg.write(':OUTPUT1:STATE OFF')
    # wait...
    time.sleep(0.1)
    # UNRUN state
    awg.write('AWGCONTROL:STOP')
    # wait...
    time.sleep(0.5)
    
    # delete any older waveform with the same name
    awg.write('WLISt:WAVeform:DELete "{0}"'.format(wfm_name))
    time.sleep(0.05)
    # check if there is any errors on AWG's buffer
    error_str = awg.query('SYSTEM:ERROR:NEXT?')
    # but discard it because it solely means it is a new waveform name
    if error_str != '0,"No error"\n':
        error_str = awg.query('SYSTEM:ERROR:NEXT?')
    
    
    # create new waveform
    awg.write('WLISt:WAVeform:NEW "{0}", {1}, INT'.format(wfm_name,len(data)))
    # download data
    awg.write_binary_values('WLISt:WAVeform:DATA "{0}",'.format(wfm_name), data, datatype='H', is_big_endian=False)
    time.sleep(0.1)
    # assign new waveform to channel 1
    awg.write('SOURce1:WAVeform "{0}"'.format(wfm_name))
    
    # check for errors
    error_str = awg.query('SYSTEM:ERROR:NEXT?')
    if error_str != '0,"No error"\n':
        print('ERROR - AWG response to error query: ' + error_str)
        fid_meas_log.write('Errors occured in AWG during waveform upload')
        fid_meas_log.write('AWG response to error query: ' + error_str)
        sys.exit('Errors occured in AWG during waveform upload')
    
    # wait...
    time.sleep(0.1)
    # AWG on RUN state
    awg.write('AWGCONTROL:RUN')
    # wait...
    time.sleep(0.5)
    # check if there is any errors on AWG's buffer
    error_str = awg.query('SYSTEM:ERROR:NEXT?')
    if error_str == '0,"No error"\n':
        # turn AWG's output 1 OFF
        awg.write(':OUTPUT1:STATE ON')
    else:
        print('AWG Output 1 not enabled')
        print('ERROR - AWG response to error query: ' + error_str)
        fid_meas_log.write('AWG Output 1 not enabled')
        fid_meas_log.write('Errors occured in AWG during waveform upload')
        fid_meas_log.write('AWG response to error query: ' + error_str)
        sys.exit('Errors occured in AWG during waveform upload')
        
    return len(data)


In [ ]:
    def initialize_w2p(self,**options):
        """Initializes the system for w2p acquisition"""
        defaults = {"reset": True, "port1": 1,"port2":2, "b_name_list": ["A", "B", "C", "D"]}
        initialize_options = {}
        for key, value in defaults.iteritems():
            initialize_options[key] = value
        for key, value in options.iteritems():
            initialize_options[key] = value
        if initialize_options["reset"]:
            self.write("SYST:FPRESET")
        b1_name = initialize_options["b_name_list"][initialize_options["port1"] - 1]
        b2_name= initialize_options["b_name_list"][initialize_options["port2"] - 1]
        # Initialize Port 1 traces A1_D1,B1_D1,B2_D1
        self.write("DISPlay:WINDow1:STATE ON")
        self.write("CALCulate:PARameter:DEFine 'A{0}_D{0}',R{0},{0}".format(initialize_options["port1"]))
        self.write("DISPlay:WINDow1:TRACe1:FEED 'A{0}_D{0}'".format(initialize_options["port1"]))
        self.write("CALCulate:PARameter:DEFine 'B{0}_D{0}',{1},{0}".format(initialize_options["port1"],b1_name))
        self.write("DISPlay:WINDow1:TRACe2:FEED 'B{0}_D{0}'".format(initialize_options["port1"]))
        self.write("CALCulate:PARameter:DEFine 'A{1}_D{0}',R{1},{0}".format(initialize_options["port1"],
                                                                            initialize_options["port2"] ))
        self.write("DISPlay:WINDow1:TRACe3:FEED 'A{1}_D{0}'".format(initialize_options["port1"],
                                                                    initialize_options["port2"]))

        self.write("CALCulate:PARameter:DEFine 'B{1}_D{0}',{2},{0}".format(initialize_options["port1"],
                                                                           initialize_options["port2"],
                                                                           b2_name))
        self.write("DISPlay:WINDow1:TRACe4:FEED 'B{1}_D{0}'".format(initialize_options["port1"],
                                                                           initialize_options["port2"]))
        # Initialize Port 2 Traces A1_D2,B1_D2,

        self.write("CALCulate:PARameter:DEFine 'A{0}_D{1}',R{0},{1}".format(initialize_options["port1"],
                                                                            initialize_options["port2"] ))
        self.write("DISPlay:WINDow1:TRACe5:FEED 'A{0}_D{1}'".format(initialize_options["port1"],
                                                                    initialize_options["port2"]))
        self.write("CALCulate:PARameter:DEFine 'B{0}_D{1}',{2},{1}".format(initialize_options["port1"],
                                                                           initialize_options["port2"],
                                                                           b1_name))
        self.write("DISPlay:WINDow1:TRACe6:FEED 'B{0}_D{1}'".format(initialize_options["port1"],
                                                                    initialize_options["port2"]))


        self.write("CALCulate:PARameter:DEFine 'A{1}_D{1}',R{1},{1}".format(initialize_options["port1"],
                                                                            initialize_options["port2"] ))
        self.write("DISPlay:WINDow1:TRACe7:FEED 'A{1}_D{1}'".format(initialize_options["port1"],
                                                                    initialize_options["port2"]))
        self.write("CALCulate:PARameter:DEFine 'B{1}_D{1}',{2},{1}".format(initialize_options["port1"],
                                                                           initialize_options["port2"],
                                                                           b2_name))
        self.write("DISPlay:WINDow1:TRACe8:FEED 'B{1}_D{1}'".format(initialize_options["port1"],
                                                                    initialize_options["port2"]))
        self.sweep_type = self.get_sweep_type()
        self.frequency_list=self.get_frequency_list()
